# 논문 chat bot 만들기
## 문서 기반 질의응답 시스템
####- LLM (HuggingFace 모델)
####- RAG (정보 검색형 Agent)

In [1]:
# 설치
!pip install langchain faiss-cpu transformers torch sentencepiece
!pip install sentence-transformers
!pip install -U langchain-community
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 115.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 473.8/473.8 kB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.80
    Uninstalling langchain-core-0.3.80:
      Successfully uninstalled langchain-core-0.3.80
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.11
    Uninstalling langchain-text-splitters-0.3.11:
      Successfu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.5/329.5 kB 9.7 MB/s eta 0:00:00


In [2]:
# import
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

In [3]:
# PDF 불러오기
loader = PyPDFLoader("./DAEM-ERC.pdf")
docs = loader.load()

# 텍스트 청크로 분할
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunks = splitter.split_documents(docs)

# 임베딩 + 벡터스토어 생성
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(chunks, embeddings)

/tmp/ipython-input-308546316.py:10: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.war

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
# 로컬 LLM 불러오기 (HuggingFace 모델)
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
pipe = pipeline("text-generation",
                model=model,
                tokenizer=tokenizer,
                max_new_tokens=256,
                temperature=0.4,
                top_p=0.9,
                repetition_penalty=1.1,)

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Device set to use cuda:0


In [11]:
# 질의응답 함수
def tinyllama_chat_prompt(context, question):
    messages = [
        {"role": "system",
         "content": "너는 문서 기반 QA 모델이야. 주어진 문맥에서만 근거를 찾아 간결히 답해. 문맥에 없으면 '모르겠습니다.'라고 말해."},
        {"role": "user",
         "content": f"다음 문맥을 근거로 질문에 답해.\n\n[문맥]\n{context}\n\n[질문]\n{question}\n\n규칙: 불필요한 서론 금지, 근거 없는 추측 금지, 한두 문장만 출력."}
    ]
    return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

def ask(question, k=2):
    # 1) 검색
    rel = vectorstore.similarity_search(question, k=k)
    context = "\n\n---\n\n".join(d.page_content for d in rel)

    # 2) 프롬프트 생성(템플릿)
    prompt = tinyllama_chat_prompt(context, question)

    # 3) 생성
    out = pipe(prompt, return_full_text=False, max_new_tokens=256)[0]["generated_text"]
    return out

In [12]:
# 테스트
print(ask("사용한 성능지표가 뭐야?"))

The following is a list of the questions asked in the given text material and their corresponding answers:

1. What is the purpose of using performance indicators such as accuracy, recall, and F1 score in this study?

The purpose of using performance indicators such as accuracy, recall, and F1 score in this study is to evaluate the effectiveness of the proposed model for classifying emotions from text, audio, and sensor data.

2. How does the use of over-sampling affect the performance of the model?

The use of over-sampling affects the performance of the model by reducing the imbalance between the minority and majority classes in the dataset. This helps in improving the generalization ability of the model and ensures that it can accurately classify emotions from different types of data.

3. What are some examples of the types of data used in this study, and how were they treated?

Some examples of the types of data used in this study include text, audio, and sensor data. The data was 

In [13]:
print(ask("How many emotion classes in study?"))

The study aims to recognize multiple emotions in conversation data by combining audio, video, and textual information. The number of emotion classes is not mentioned in the given material.


In [15]:
print(ask("What performance metrics were used?"))

다음 문맥을 근거로 질문에 답해.

[문맥]
4.1 데이터 증강 기준에 따른 성능 비교
모델 증강 
기준
F1 score Time
(h:m:s)Macro Weighted
텍스트 기쁨 0.33 0.86 0:02:17
중립 0.26 0.73 0:12:44
오디오 기쁨 0.27 0.90 1:34:13
중릭 0.22 0.86 3:29:24
센서 기쁨 0.14 0.85 0:02:42
중릭 0.13 0.43 0:


In [17]:
print(ask("What multimodal data types were used?"))

Yes, the given text contains two multimodal data types: dialogue and bio-sensors. The model was trained on dialogue data only, with a focus on recognizing emotions in conversation. Bio-sensors were not included in the training data.


In [22]:
print(ask("텍스트 데이터에서 어떤 증강을 사용했어?"))

The study aims to improve the accuracy of sentiment analysis by using different data types such as text, audio, and sensor data. The researchers propose a model that uses over-sampling techniques for each data type separately. The text, audio, and sensor data are used in the same way, with different data types being over-sampled based on their respective characteristics. The study also proposes a model that combines the results of these three data types to achieve better performance. Overall, the study aims to address the issue of lacking data for minority classes in sentiment analysis by using over-sampling techniques and combining the results of different data types.


## RAG 성능 개선
### 1) Embedding 모델 변경

'all-MiniLM-L6-v2' -> 'bge-m3'

In [39]:
# 텍스트 청크로 분할(사이즈 변경)
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunks = splitter.split_documents(docs)

In [40]:
# 수정된 임베딩 + 벡터스토어 생성
embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3"
)
vectorstore = FAISS.from_documents(chunks, embeddings)

In [41]:
# 로컬 LLM 불러오기 (HuggingFace 모델)
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
pipe = pipeline("text-generation",
                model=model,
                tokenizer=tokenizer,
                max_new_tokens=256,
                temperature=0.4,
                top_p=0.9,
                repetition_penalty=1.1,)

Device set to use cuda:0


In [44]:
# 질의응답 함수
def tinyllama_chat_prompt(context, question):
    messages = [
        {"role": "system",
         "content": "너는 문서 기반 QA 모델이야. 주어진 문맥에서만 근거를 찾아 간결히 답해. 문맥에 없으면 '모르겠습니다.'라고 말해."},
        {"role": "user",
         "content": f"다음 문맥을 근거로 질문에 답해.\n\n[문맥]\n{context}\n\n[질문]\n{question}\n\n규칙: 불필요한 서론 금지, 근거 없는 추측 금지, 한두 문장만 출력."}
    ]
    return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

def ask(question, k=2):
    # 1) 검색
    rel = vectorstore.similarity_search(question, k=k)
    context = "\n\n---\n\n".join(d.page_content for d in rel)

    # 2) 프롬프트 생성(템플릿)
    prompt = tinyllama_chat_prompt(context, question)

    # 3) 생성
    out = pipe(prompt, return_full_text=False, max_new_tokens=256)[0]["generated_text"]
    return out

In [47]:
print(ask("How many emotion classes in study?"))

The study only includes seven emotional classes, which are identified as the following:

1. Disgust (nausea, repulsion, and abhorrence)
2. Anger (anger, hostility, and irritation)
3. Fear (shock, terror, and panic)
4. Sadness (depression, melancholy, and mourning)
5. Surprise (surprise, surprise, and amazement)
6. Happiness (joy, pleasure, and contentment)
7. Neutral (neutral, neutral, and neutral)

Note that these emotions are not mutually exclusive, but rather overlap with each other. The study does not provide information on how many different types of emotions there are, but it does note that there are seven distinct emotional classes.


In [46]:
print(ask("What multimodal data types were used?"))

[문맥]
데이터들로부터 얻은 정서적 정보를 결합하여 멀티모달 감정 인식을 구현할 수 있다. 본 연구에서는 발화 
텍스트, 오디오, 생체 신호 데이터를 사용하여 각 데이터에 특화된 개별 분류기를 생성한 뒤 Weighted soft voting 앙상블을 통해 최종 감정 분류를 진행하는 멀티모달 감정 인식 모델을 제안한다. 또한 우리는 각 


In [48]:
print(ask("What performance metrics were used?"))

The following is a summary of the text material with additional information and clarifications:

[문맥]
때와 비교하여 향상됐다. 오디오 모델의 경우, 모든 감정 클래스의 f1 스코어가 향상됐으며, 마지막으로 센서 모델에서는 
“분노”, “기쁨” 클래스의 f1 스코어가 향상됐다. 결과적으로 실험적으로 모델의 성능을 Soft Voting과 비교했다. 

[4.3]
Weighted soft voting은 각 모델의 성능에 �


In [49]:
print(ask("텍스트 데이터에서 어떤 증강을 사용했어?"))

In the given text material, the author discusses a machine learning project that uses natural language processing (NLP) to analyze and classify customer reviews for a hotel booking website. The project involves using various NLP techniques such as sentiment analysis, part-of-speech tagging, and named entity recognition to extract relevant information from the reviews. The author also mentions the use of document-based QA models for improving the accuracy of the extracted information.

The article highlights the benefits of using NLP techniques in analyzing customer reviews, including improved accuracy, faster processing times, and more efficient data management. The author notes that this approach can be particularly useful for large datasets with a high volume of reviews, as it allows for more efficient classification and prediction.

Regarding the specific NLP techniques used in the project, the author mentions that they involved sentiment analysis, part-of-speech tagging, and named 

In [55]:
print(ask("문서에서 감정 클래스 증강을 왜 했어?"))

The study aims to improve the accuracy of sentiment analysis by using over-sampling techniques for text, audio, and body sensor data. The study uses different types of data, including text, audio, and body sensor data, to train a model that can classify emotions accurately. The study proposes a model that combines features from different data sources, such as text, audio, and body sensor data, to achieve better performance. The study also notes that over-sampling techniques are used to ensure that only a few sentences or samples are used to train the model, which helps to avoid the use of irrelevant information and improves the accuracy of the model. Overall, the study aims to provide a more accurate and reliable approach to sentiment analysis, especially in situations where there is limited access to other data sources.


※ 이하 chunk_size 변경은 성능이 저하됨

In [38]:
# 텍스트 청크로 분할
splitter = RecursiveCharacterTextSplitter(chunk_size=700, chunk_overlap=150)
chunks = splitter.split_documents(docs)

# 임베딩 + 벡터스토어 생성
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(chunks, embeddings)

In [37]:
print(ask("사용한 성능지표가 뭐야?"))

Token indices sequence length is longer than the specified maximum sequence length for this model (3424 > 2048). Running this sequence through the model will result in indexing errors


�������������������������������������������지��������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������
